In [30]:
import json
import os
from elasticsearch import Elasticsearch

FILE = os.environ['DATA_PATH']
ES_URL = 'http://192.168.11.2:9201'
_scrollId = ''
_cArray = []


def _convertJsonToString(jHits):
    for _hits in jHits:
        _cArray.append(json.dumps(_hits['_source']))
    

# 0. setup ES
es = Elasticsearch(hosts=[ES_URL])

# 1. run search with scroll
res = es.search(index='movies_4_blogs', doc_type='_doc', scroll='1m', size=1000)
_scrollId = res['_scroll_id']
_jContent = res['hits']['hits']

# 2. convert json to string 
_convertJsonToString(_jContent)

# 3. loop the scroll till end of time...
while 1==1:
    res = es.scroll(scroll_id=_scrollId, scroll='1m')
    _scrollId = res['_scroll_id']
    _jContent = res['hits']['hits']
    
    # break condition
    if len(_jContent) == 0:
        break
    
    _convertJsonToString(_jContent)

# x. clear the last scroll
es.clear_scroll(scroll_id=_scrollId)

# 4. write to file
file = open(FILE, 'a')
for _json in _cArray:
    file.write(_json)
    file.write('\n')

print('done~ writing ', len(_cArray), ' lines of movie data')






done~ writing  146178  lines of movie data
